
# Atrial fiber
This examples shows how to generate fibers with Laplace-Dirichlet-Rule-Based-Method


## Perform the required imports
Import the required modules and set relevant paths, including that of the working
directory, model, and ls-dyna executable (uses DEV-104373-g6d20c20aee).



In [ ]:
import os

import numpy as np
import pyvista as pv

import ansys.heart.core.models as models
from ansys.heart.simulator.simulator import BaseSimulator, DynaSettings

# specify the path to the working directory and heart model
workdir = os.path.abspath(os.path.join("downloads", "Strocchi2020", "01", "FourChamber"))


path_to_model = os.path.join(workdir, "heart_model.vtu")

# specify LS-DYNA path
lsdyna_path = r"ls-dyna_smp"

# load heart model.
model: models.FourChamber = models.FourChamber(working_directory=workdir)
model.load_model_from_mesh(path_to_model, path_to_model.replace(".vtu", ".partinfo.json"))

## Instantiate the simulator object
instantiate simulator. Change options where necessary.



<div class="alert alert-info"><h4>Note</h4><p>The DynaSettings object supports several LS-DYNA versions and platforms.
   Including: "smp", "intempi", "msmpi", "windows", "linux", or "wsl" Choose
   the one that is appropriate for you.</p></div>



In [ ]:
# instantiate LS-DYNA settings of choice
dyna_settings = DynaSettings(
    lsdyna_path=lsdyna_path, dynatype="intelmpi", num_cpus=4, platform="windows"
)


simulator = BaseSimulator(
    model=model,
    dyna_settings=dyna_settings,
    simulation_directory=os.path.join(workdir, "simulation"),
)

simulator.settings.load_defaults()

# remove fiber/sheet information if already exists
model.mesh.cell_data["fiber"] = np.zeros((model.mesh.n_cells, 3))
model.mesh.cell_data["sheet"] = np.zeros((model.mesh.n_cells, 3))

## Compute atrial fibers



In [ ]:
# Compute left atrium fiber
la = simulator.compute_left_atrial_fiber()

# Appendage apex point should be manually given to compute right atrium fiber
appendage_apex = [-33, 82, 417]
ra = simulator.compute_right_atrial_fiber(appendage_apex)

<div class="alert alert-info"><h4>Note</h4><p>You may need to define an appropriate point for the right atrial appendage
   the list defines the x, y, and z coordinates close to the appendage.</p></div>



## Plot bundle selection results



In [ ]:
la.set_active_scalars("bundle")
la.plot()

<img src="file://_static/images/la_bundle.png" width="400pt" align="center">



In [ ]:
ra.set_active_scalars("bundle")
ra.plot()

<img src="file://_static/images/ra_bundle.png" width="400pt" align="center">



## Plot fibers



In [ ]:
plotter = pv.Plotter()
mesh = la.ctp()
streamlines = mesh.streamlines(vectors="e_l", source_radius=50, n_points=50000)
tubes = streamlines.tube()
plotter.add_mesh(mesh, opacity=0.5, color="white")
plotter.add_mesh(tubes, color="red")
plotter.show()

<img src="file://_static/images/la_fiber.png" width="400pt" align="center">



In [ ]:
plotter = pv.Plotter()
mesh = ra.ctp()
streamlines = mesh.streamlines(vectors="e_l", source_radius=50, n_points=50000)
tubes = streamlines.tube()
plotter.add_mesh(mesh, opacity=0.5, color="white")
plotter.add_mesh(tubes, color="red")
plotter.show()

<img src="file://_static/images/ra_fiber.png" width="400pt" align="center">



In [ ]:
# Atrial fibers are automatically assigned to heart model after computation.

plotter = pv.Plotter()
mesh = model.mesh.ctp()
streamlines = mesh.streamlines(vectors="fiber", source_radius=100, n_points=50000)
tubes = streamlines.tube()
plotter.add_mesh(mesh, opacity=0.5, color="white")
plotter.add_mesh(tubes, color="red")
plotter.show()

<img src="file://_static/images/atrial_fiber_assign.png" width="400pt" align="center">

